# Project management

In [35]:
from os import path, getenv
from mlrun import new_project

project_name = '-'.join(filter(None, ['properties-management']))
project_path = path.abspath('conf')
project = new_project(project_name, project_path, init_git=True)

print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/igztraining/mlrun/Day-4/conf
Project name: properties-management


In [36]:
from mlrun import run_local, NewTask, mlconf, import_function, mount_v3io

# Target location for storing pipeline artifacts
artifact_path = path.abspath('jobs')
# MLRun DB path or API service URL
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Artifacts path: /User/igztraining/mlrun/Day-4/jobs
MLRun DB path: http://mlrun-api:8080


# Deploy code as an MLRun function

In [37]:
from mlrun import code_to_function, mlconf,mount_v3io

In [38]:
fn = code_to_function(project='Iguazio-Training',name='properities-handler', kind='job',filename='modules/mlrun_functions.ipynb')

In [39]:
fn.spec.image = "mlrun/mlrun"

In [40]:
fn.apply(mount_v3io())

# Add function to project

In [45]:
project.set_function(fn,name='properties-handler')

In [46]:
project.functions

[{'name': 'properties-handler',
  'spec': {'kind': 'job',
   'metadata': {'name': 'properities-handler',
    'tag': '',
    'project': 'properties-management'},
   'spec': {'command': '',
    'args': [],
    'image': 'mlrun/mlrun',
    'env': [{'name': 'V3IO_API', 'value': ''},
     {'name': 'V3IO_USERNAME', 'value': ''},
     {'name': 'V3IO_ACCESS_KEY', 'value': ''}],
    'default_handler': '',
    'entry_points': {'save_property': {'name': 'save_property',
      'doc': '',
      'parameters': [{'name': 'context', 'default': ''},
       {'name': 'source_file', 'default': ''},
       {'name': 'target_file', 'default': ''}],
      'outputs': [{'default': ''}],
      'lineno': 8},
     'filter_properties': {'name': 'filter_properties',
      'doc': '',
      'parameters': [{'name': 'context', 'default': ''},
       {'name': 'source_data', 'type': 'DataItem', 'default': ''},
       {'name': 'target_dataset', 'default': ''},
       {'name': 'city', 'default': ''}],
      'outputs': [{'defa

### Run interactive

In [47]:
params = {"source_file" : "/User/igztraining/mlrun/Day-4/data/demo.csv",
"target_file" : "/User/igztraining/mlrun/Day-4/data/condos.csv" }

In [48]:
save_properties = project.func('properties-handler').run(name='run_withcode',artifact_path="/User/artifacts",handler='save_property',params=params,project='Iguazio-Training')

> 2020-09-02 12:50:29,553 [info] starting run run_withcode uid=d40a506c6604436bbf63276fea8a9116  -> http://mlrun-api:8080
> 2020-09-02 12:50:29,750 [info] Job is running in the background, pod: run-withcode-5cx44
/User/igztraining/mlrun/Day-4/data/demo.csv
> 2020-09-02 12:50:34,468 [info] run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...ea8a9116,0,Sep 02 12:50:34,completed,run_withcode,v3io_user=marcelokind=jobowner=marcelohost=run-withcode-5cx44,,source_file=/User/igztraining/mlrun/Day-4/data/demo.csvtarget_file=/User/igztraining/mlrun/Day-4/data/condos.csv,properties_available=54,condo_properties


to track results use .show() or .logs() or in CLI: 
!mlrun get run d40a506c6604436bbf63276fea8a9116 --project Iguazio-Training , !mlrun logs d40a506c6604436bbf63276fea8a9116 --project Iguazio-Training
> 2020-09-02 12:50:35,893 [info] run executed, status=completed


### Run step 2 with Step 1's output

In [49]:
params['city']="SACRAMENTO"
params['target_dataset']="sacramento_condos"

In [50]:
sp_run = project.func('properties-handler').run(name='run_withcode',artifact_path="/User/artifacts",handler='filter_properties',
                inputs={"source_data":save_properties.outputs["condo_properties"]},
                params=params,project='Iguazio-Training')

> 2020-09-02 12:50:45,973 [info] starting run run_withcode uid=2e998b26bd104908b6e73bf056117f5a  -> http://mlrun-api:8080
> 2020-09-02 12:50:46,138 [info] Job is running in the background, pod: run-withcode-58nkj
> 2020-09-02 12:50:50,592 [info] run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...56117f5a,0,Sep 02 12:50:50,completed,run_withcode,v3io_user=marcelokind=jobowner=marcelohost=run-withcode-58nkj,source_data,source_file=/User/igztraining/mlrun/Day-4/data/demo.csvtarget_file=/User/igztraining/mlrun/Day-4/data/condos.csvcity=SACRAMENTOtarget_dataset=sacramento_condos,,sacramento_condos


to track results use .show() or .logs() or in CLI: 
!mlrun get run 2e998b26bd104908b6e73bf056117f5a --project Iguazio-Training , !mlrun logs 2e998b26bd104908b6e73bf056117f5a --project Iguazio-Training
> 2020-09-02 12:50:52,274 [info] run executed, status=completed


In [51]:
project.save()

# Run function as part of a pipeline

In [52]:
import kfp
from kfp import dsl
from mlrun import run_pipeline

In [53]:
@dsl.pipeline(
    name = 'Property Management',
    description = 'Condo reviews'
)

# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        f.apply(mount_v3io())


def job_pipeline(
    source_file :str = ""
) -> None:
    
    save_properties = project.func('properties-handler').as_step(handler='save_property',
                            params={"source_file" : source_file,
                                    "target_file" : "/User/igztraining/mlrun/data/condos.csv" },
                            outputs=['condo_properties'])
    
    
    filter_properties = project.func('properties-handler').as_step(handler='filter_properties',
                                   params={"city" : "SACRAMENTO",
                                           "target_dataset" : "sacramento_condos" },
                                   inputs={"source_data":save_properties.outputs["condo_properties"]})

In [54]:
kfp.compiler.Compiler().compile(job_pipeline, 'jobpipe.yaml')
artifact_path = 'v3io:///users/admin/kfp/{{workflow.uid}}/'
arguments = {"source_file" : "/User/igztraining/mlrun/Day-4/data/demo.csv"}
run_id = run_pipeline(job_pipeline, arguments, experiment='property_management', artifact_path=artifact_path)

> 2020-09-02 12:53:24,062 [info] Pipeline run id=ff92a738-2a05-447a-aee5-6f7998ee3eb9, check UI or DB for progress
